In [32]:
# we could use also tensor flow, there are multiple implementations of word2vec
from gensim.models import word2vec
import pandas as pd
import json
from pathlib import Path
import string
import re
import pickle

In [2]:
#word to vec exploration  

## .strip(string.punctuation).lower()


In [3]:
#first load all recipe data
BOrecipes = []
BOingredients = []
BOids = []
BOpath = Path('~/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/')
for i, po in enumerate(BOpath.expanduser().glob('*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        BOrecipes.append(r)
        BOing = r.get('Ingredients')
        BOid = r.get('RecipeID')
        BOids.append(id)
        if BOing is not None:
            #BOingredients.append([i['Name'].strip(string.punctuation).lower().strip(string.digits).strip() for i in BOing if i is not None and i['Name'] is not None])
            BOingredients.append([re.sub(r'[^a-z ]', '', i['Name'].lower().strip()) for i in BOing if i is not None and i['Name'] is not None])

/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-68-87332.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-71-87778.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-19-156227.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-127-74-2431270.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-112-159-862538.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-62-166860.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-35-191-157959.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-76-83346.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-74-82-565669.json
/Users/bridgitboulahanis/Dropbox

In [4]:
#now we want to load spoonacular data and create a dataframe that has ingredients and their aisle
SArecipes = []
SAingredients = []
SAingclass = []
SAnoise = []
SApath = Path('~/Dropbox/Not_work/insight/greenmarkets/spoonac/data/')
for i, po in enumerate(SApath.expanduser().glob('*/*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        for rec in r:
            currenting = rec['extendedIngredients']
            for ing in currenting:
                SAingredients.append(ing['name'])
                SAingclass.append(ing['aisle'])  
                #SAnoise.extend(ing['metaInformation'])
                SAnoise.extend(i.strip() for i in ing['metaInformation'])

                    





In [54]:
#grab all the ingrdients from spoonacular and put in dataframe
classifying = pd.DataFrame({'ingredient': SAingredients, 'aisle': SAingclass})
classifying.drop_duplicates(inplace=True)
classifying

,ingredient,aisle
0,bell pepper,Produce
1,canned coconut milk,Canned and Jarred
2,carrots,Produce
3,cornstarch,Baking
4,curry powder,Spices and Seasonings
...,...,...
211971,aji amarillo chile paste,Ethnic Foods
211986,braise,?
212025,seasalt,Spices and Seasonings
212028,camembert cheese,Cheese


In [66]:
#just for looking at classifying, not a prod cell
classifying.dropna
classifying

#have 4533 data

#classifying['aisle'].nunique()
classifying.aisle.unique()

array(['Produce', 'Canned and Jarred', 'Baking', 'Spices and Seasonings',
       'Ethnic Foods;Spices and Seasonings',
       'Canned and Jarred;Produce;Ethnic Foods',
       'Bakery/Bread;Ethnic Foods', 'Meat', 'Pasta and Rice', 'Beverages',
       'Pasta and Rice;Ethnic Foods;Health Foods',
       'Oil, Vinegar, Salad Dressing', 'Seafood', 'Ethnic Foods',
       'Condiments', 'Canned and Jarred;Milk, Eggs, Other Dairy',
       'Pasta and Rice;Canned and Jarred',
       'Produce;Spices and Seasonings', 'Milk, Eggs, Other Dairy',
       'Pasta and Rice;Health Foods', 'Refrigerated;Produce;Ethnic Foods',
       'Nuts;Savory Snacks', 'Produce;Ethnic Foods',
       'Produce;Ethnic Foods;Spices and Seasonings',
       'Health Foods;Baking', '?', 'Nuts',
       'Gluten Free;Health Foods;Baking', 'Bakery/Bread',
       'Gluten Free;Ethnic Foods;Health Foods', 'Cheese',
       'Ethnic Foods;Health Foods', None, 'Savory Snacks',
       'Nut butters, Jams, and Honey;Health Foods', 'Frozen',
   

In [82]:
#also not a prod cell
classifying.loc[classifying['aisle'] == 'Canned and Jarred;Produce;Ethnic Foods']


#okay so steps include more deduping of the classification matrix
# show necessary here: classifying.loc[classifying['aisle'] == 'Canned and Jarred;Produce;Ethnic Foods']

#then iteratively grab the spoon suggestions for each ingredient
#input = ingredient, output = suggestion, features = aisle, meta info, possible units

#then probably making some executive decisions for cases of multiple aisles
#but really with only 100 that isn't so hard



#THEN adding features --> start with the w2v outputs n=10? and see if you can predict aisle


,ingredient,aisle
7,jalapeno,Canned and Jarred;Produce;Ethnic Foods
526,jalapeno pepper,Canned and Jarred;Produce;Ethnic Foods
1331,jalapenos,Canned and Jarred;Produce;Ethnic Foods
2229,jalapeno peppers,Canned and Jarred;Produce;Ethnic Foods
3794,jalapeno chiles,Canned and Jarred;Produce;Ethnic Foods
6453,jalapeno chile,Canned and Jarred;Produce;Ethnic Foods
11607,jalapeno chili,Canned and Jarred;Produce;Ethnic Foods
16744,jalapeño,Canned and Jarred;Produce;Ethnic Foods
24964,jalepeno,Canned and Jarred;Produce;Ethnic Foods
42019,bottled ginger,Canned and Jarred;Produce;Ethnic Foods


In [7]:
#some words you noticed should get added to SAnoise
wta = ['ground', 'taste', 'ounce']
SAnoise.extend(wta)
SAnoise = set(SAnoise)
#remember you can check if it is in there with: 'fresh' in SAnoise

In [8]:
#suite of deduping functions

#on loading we tokenize, strip punctuation&digits&spaces and make lowercase 

def removenoise(input_ing):
    noise_free_ing = []
    for word in input_ing:
        checked = []
        splitit = word.split()
        checked.extend(i for i in splitit if i not in SAnoise)
        noise_free_ing.append(' '.join(checked))
    return noise_free_ing
        
#stem? lemmatize? part of speech tagging? stop words
    
    

In [9]:
#call the dedupe functions
nning = []
for i in BOingredients:
    nning.append(removenoise(i))



In [58]:
# classify ingredients into aisles
whichaisle = {}
seen = set()
notin = []
for i in nning:
    for j in i:
        if j in seen:
            continue
        seen.add(j) 
        a = classifying.index[classifying['ingredient'] == j].tolist()
        if len(a) >= 1:
            whichaisle[j] = classifying['aisle'][a].tolist()
        if len(a) == 0:
            whichaisle[j] = None
            notin.append(j)

            # okay so instead of exact matching maybe fuzzy wuzzy
            # also do another call to get more data

In [61]:
print(len(whichaisle))
notin[10:]

39506


['thai chilies',
 'thai basil leaves',
 'old el paso mild enchilada sauce',
 'wheat tortillas',
 'precooked chicken breast',
 'reduced fat mexican cheese',
 'salt pepper',
 'calypso broth',
 'medium shrimp',
 'raspberry balsamic vinegar',
 'chambord raspberry liqueur',
 'paprika and pepper',
 'reduced sodium',
 '',
 'wholeberry cranberry sauce',
 'bonefish grill limetomatogarlic sauce',
 'roughly sundried tomatoes',
 'roughly tomatoes',
 'tomatoe paste',
 'garlic inch cube of ginger about grams',
 'cube of ginger about grams',
 'of baby corn cobs',
 'peas or vegetables of your choice',
 'the spice blend',
 'pepper teaspoon cumin teaspoon gro',
 'wholewheat spaghetti',
 'steak mince',
 'garlic powdergranules',
 'sea salt flakes',
 'beef or chicken',
 'taco seasoning or southwest seasoning',
 'corn or flour tortillas',
 'colby jack or cheddar cheese',
 'baby rays bbq',
 'tfrozen southernstyle hash brown potatoes',
 'ranch or cheese dressing',
 'corn flake crumbs or panko crumbs',
 'butte

In [33]:
#Actual W2V modeling
# Set values for NN parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 3    # 50% of the corpus                    
num_workers = 4       # Number of CPUs
context = 10          # Context window size; 
downsampling = 1e-3   # threshold for configuring which 
                    # higher-frequency words are randomly downsampled

# Initialize and train the model 
model = word2vec.Word2Vec(nning, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

In [43]:
#output both the w2v model and the aisle classification dict
pickle.dump(model,open("bigoven/w2vmodel.pkl","wb"))
pickle.dump(whichaisle,open("bigoven/aisleclassification.pkl","wb"))
pickle.dump(SAnoise, open('bigoven/noiselist.pkl', 'wb'))

In [41]:
#proof of concept, also not a production cell
print(model.wv.most_similar('feta cheese'))
print('=============')
print(model.wv.most_similar('baby spinach'))

[('kalamata olives', 0.9773502349853516), ('baby spinach', 0.9714789986610413), ('plum tomatoes', 0.9671364426612854), ('pine nuts', 0.9558079838752747), ('sundried tomatoes', 0.952326774597168), ('grape tomatoes', 0.9518359899520874), ('basil leaves', 0.9489792585372925), ('cherry tomatoes', 0.9453437924385071), ('san marzano tomatoes', 0.9453309774398804), ('goat cheese', 0.9413638114929199)]
[('sundried tomatoes', 0.9894789457321167), ('goat cheese', 0.9803232550621033), ('grape tomatoes', 0.9769477844238281), ('kalamata olives', 0.9761931300163269), ('feta cheese', 0.9714789986610413), ('artichoke hearts', 0.9684235453605652), ('pine nuts', 0.9621485471725464), ('orzo pasta', 0.9534425735473633), ('grilled chicken breasts', 0.9517673850059509), ('oil cured olives', 0.9511600732803345)]
